In [1]:
# coding: utf-8

from sklearn import preprocessing
from time import time
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import silhouette_samples, silhouette_score
from operator import truediv
from sklearn.metrics import pairwise_distances
import pandas as pd

get_ipython().magic('matplotlib')

df_all=pd.read_csv('diabetic_data_processed_withweight.csv',';')

df_grouped=df_all.groupby(['patient_nbr'], sort=True).last()
df_grouped = df_grouped[df_grouped['readmitted']>0]

print("Shape grouped: ",df_grouped.shape)

#print(df_grouped)

msk = np.random.rand(len(df_grouped)) < 0.8
df, df_test = df_grouped[msk].copy(deep = True), df_grouped[~msk].copy(deep = True)
df = df.reset_index()
df_test = df_test.reset_index()
print("df.shape",df.shape)
print("df_test.shape",df_test.shape)

print("Columns: ",df.columns)
y_train=df['readmitted']
y_test=df_test['readmitted']

print("Test", set(y_test))

# In[112]:

to_del = ['admission_source_id','encounter_id', 'patient_nbr','medical_specialty','payer_code','index','admission_type_id','discharge_disposition_id',
          'nateglinide','chlorpropamide','acetohexamide','tolbutamide','acarbose','miglitol','troglitazone','tolazamide','examide','citoglipton','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone']        
print (to_del)

#Filter_selected cols
filtered_cols = [c for c in df.columns if (c not in to_del) ]#and ('ENF' not in c)
df_2 = df[filtered_cols]
df_2_test = df_test[filtered_cols]
print ("df_2",df_2.shape)

from sklearn.model_selection import train_test_split
import numpy as np
df_3 = df_2.fillna(value=np.mean(df_2,axis=0),inplace=False,axis=0).values
df_3_test = df_2_test.fillna(value=np.mean(df_2,axis=0),inplace=False,axis=0).values
print ("df_3",df_3.shape)
print(df_2.columns)
x_train=df_3[:,0:-1]
x_test=df_3_test[:,0:-1]

print(set(y_train))
print(set(y_test))
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

Using matplotlib backend: Qt5Agg
Shape grouped:  (17167, 49)
df.shape (13635, 50)
df_test.shape (3532, 50)
Columns:  Index(['patient_nbr', 'encounter_id', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'dia

In [3]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB

rf_pipeline = Pipeline([
                        ('std', preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True)), 
                        ('smt', SMOTE(n_jobs=-1, random_state=42)),
                        ('classifier', GaussianNB())
                       ])

tuned_parameters = {  
}  

#scores = ['precision', 'recall', 'f1']
#scores = [ 'f1_weighted','f1_micro','f1_macro','accuracy','roc_auc']
scores=['f1_weighted']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    clf = GridSearchCV(estimator=rf_pipeline , param_grid=tuned_parameters, cv=10,scoring='%s' % score,n_jobs= -1)
    clf.fit(x_train, y_train)
    print("Best parameters set found on development set:")
    print()
    #print(clf.best_params_)
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_train2, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_train2, y_pred))
    print("Accuracy score: ", accuracy_score(y_test, y_pred))
    print()

# Tuning hyper-parameters for f1_weighted

Best parameters set found on development set:

{}

Grid scores on development set:

0.509 (+/-0.034) for {}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

          29       0.21      0.66      0.32       687
          31       0.83      0.39      0.53      2845

    accuracy                           0.44      3532
   macro avg       0.52      0.53      0.42      3532
weighted avg       0.71      0.44      0.49      3532

Accuracy score:  0.4447904869762174

